In [ ]:
#Imports

#Pandas: Reading and analyzing data
import pandas as pd
#Numerical calcuations
import numpy as np
#statistical data visualization
import seaborn as sns
#Use Dates in Datetime Format
import datetime
#Tensorflow
import tensorflow as tf
#Keras: Open-Source deep-learning library 
from tensorflow import keras
#Building blocks of NN in Keras
from tensorflow.keras import layers
#EarlyStop to stop training early
from tensorflow.keras.callbacks import EarlyStopping
#Functional API: Layers for different models
from keras.layers import Dense, LSTM, Dropout
#Normalization
from sklearn.preprocessing import MinMaxScaler
#Standardization
from sklearn.preprocessing import StandardScaler
#Evaluate models
import math
#Evaluate MSE
from sklearn.metrics import mean_squared_error
#plot numpy array
import matplotlib.pyplot as plt
#Create Folder for modelling checkpoint
import os
#Callback to logg model fitting time
import time
from timeit import default_timer as timer
#Clear output after training
import IPython
import IPython.display
#Normalization
from sklearn.preprocessing import MinMaxScaler
#Standardization
from sklearn.preprocessing import StandardScaler

#Helper Class (Export Notebook as .py)
from windowgenerator import WindowGenerator

In [ ]:
# Create a function to implement a ModelCheckpoint callback with a specific filename 

def create_model_checkpoint(model_name, save_path):
    return tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(save_path), # create filepath to save model
        verbose=0, # only output a limited amount of text
        save_best_only=True) # save only the best model to file

#Helper function for time callback to log training time per epoch
class timecallback(tf.keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append((epoch, timer()-self.starttime))
        
timetaken = timecallback()

In [1]:
def compile_and_fit(model, window, model_name, client_name, MAX_EPOCHS, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[
                    tf.keras.metrics.RootMeanSquaredError(), 
                    tf.keras.metrics.MeanAbsolutePercentageError(),
                    tf.keras.metrics.MeanAbsoluteError(),
                ])

    history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping, create_model_checkpoint(
                          model_name=model.name, 
                          save_path=f"model_experiments/Local/{model_name}/{client_name}"
                      ), timetaken])
    
    return history

In [ ]:
def build_LSTM_model(
    input_shape, num_LSTM_cells, num_LSTM_layers, num_LSTM_dense_layers, num_LSTM_dense_units, LSTM_dropout,
    output_steps, num_features, model_name
):
   #Input dimensions are (N x n_past x #features)
    inputs = keras.Input(shape=input_shape, name="Input")
    #Hidden LSTM Layers
    x = layers.LSTM(num_LSTM_cells, return_sequences=True)(inputs)

    for _ in range(num_LSTM_layers):
        x = layers.LSTM(num_LSTM_cells, return_sequences=True)(x)

    x = layers.LSTM(num_LSTM_cells, return_sequences=False)(x)

    #Dropout and Dense Layers
    for _ in range(num_LSTM_dense_layers):
        x = layers.Dense(num_LSTM_dense_units, activation="relu")(x)
        x = layers.Dropout(LSTM_dropout)(x)

    #Output
    x = layers.Dense(output_steps*num_features, kernel_initializer=tf.initializers.zeros())(x)
    outputs = layers.Reshape([output_steps, num_features])(x)
    
    multi_lstm_model = keras.Model(inputs=inputs, outputs=outputs, name=model_name)

    return multi_lstm_model

def build_CNN_model(
    input_shape, conv_width, num_CNN_layers, num_CNN_filters, num_CNN_dense_layers, 
    num_CNN_dense_units, CNN_dropout, output_steps, num_features, model_name
):
   #Input dimensions are (N x n_past x #features)
    inputs = keras.Input(shape=input_shape, name="Input")
    #Hidden LSTM Layers
    x = layers.Lambda(lambda x: x[:, -conv_width:, :])(inputs)

    for _ in range(num_CNN_layers):
        x = layers.Conv1D(num_CNN_filters, kernel_size=(conv_width), activation='relu')(x)

    for _ in range(num_CNN_dense_layers):
        x = layers.Dense(num_CNN_dense_units, activation="relu")(x)
        x = layers.Dropout(CNN_dropout)(x)

    x = layers.Dense(output_steps*num_features, kernel_initializer=tf.initializers.zeros())(x)
    outputs = layers.Reshape([output_steps, num_features])(x)
    #Create model
    multi_conv_model = keras.Model(inputs=inputs, outputs=outputs, name=model_name)

    return multi_conv_model

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, 
                            dropout, mlp_dropout, out_steps, num_features, model_name
                           ):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    
    x = layers.Dense(out_steps*num_features, kernel_initializer=tf.initializers.zeros())(x)
    outputs = layers.Reshape([out_steps, num_features])(x)
    #outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs, name=model_name)